# 5. Feature Engineering, Feature Selection, and Baseline Benchmark

In [1]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import warnings
import numpy as np

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [2]:
#na_filter set to False as otherwise empty strings are interpreted as NaN
df_tweets_cleaned = pd.read_csv('..\data\Tweets_cleaned.csv', encoding='utf-8', na_filter= False)

## Feature Engineering
Let's generate some features we could possibly use. Some features, such as `emojis_flag`, `emoticons_flag`, and `hashtags_flag` are already generated. Below are some of the features we are engineering/generating:

1. `emojis_num` denotes the number of emojis used in a tweet. 
2. `emoitcons_num` denotes the number of emoticons used in a tweet. 
3. `hashtag_num` denotes the number of hashtags used in a tweet.
4. `numbers_flag` denotes whether the tweet contains numbers or not (either in Arabic or English)
5. `numbers_num` denotes the number of times a tweet contains numbers
We noticed that numbers were used in quite a few negative tweets, such as hours, time, dollars, flight numbers, etc. This is why we are generating a binary flag, as well as a numeric count of numbers used in a tweet.
6. `char_length_original` denotes the length of the the user's original tweet. This includes everything (@ mentions, RT retweets, hyperlinks, etc.)
7. `char_length_user` denotes the length of the user's cleaned tweet. The length will be based off the column `text_cleaned`
We also noticed that negative tweets were, on average, longer than positive tweets in terms of character length.
8. `mentions_num` denotes the number of mentions a tweet has (@ mentions)
9. `retweet_flag` denotes if the user's tweet retweeted a tweet (normally the retweet is one of an airline, rarely another user).
No need to create a count for retweets in a user's tweet because it's always 1.
10. `http_flag` denotes if the user's tweet has a HTTP link. 
No need to create a count for http links in a user's tweet because it's always 1 too.

The True/Flase `_flag` will need to be converted into binary flags instead (i.e. True/False into 1/0).

Any of the `_num` columns will likely need to be scaled to a scale from 0 to 1. 

We will also need to vectorize the words in the tweets. To do so, there are several ways of doing so. We could use `word2vec`, `emoji2vec`, or a combination of both of them called `phrase2vec`.

Lastly, we will need to convert airline_sentiment into 0 or 1. In this situation, because we care about classifying negative sentiment tweets, and not really care about whether it's positive or neutral, we decided to group the positive and neutral tweets as `non-negative`. All `non-negative` tweets are class `0`, whereas all `negative` tweets are class `1`.

### Generate columns `emojis_num`, `emoticons_num`, and `hashtag_num`
Generate basic features such as `emojis_num`, `emoticons_num`, `hashtag_num` from already developed columns.

In [3]:
#creates emojis_num column
def create_emojis_num(df):
    df['emojis_num'] = 0 
    
    for i, row in df.iterrows():    
        if df.at[i, 'emojis_flag']:
            tweet_emojis = df.at[i, 'emojis']
            #strip brackets, quote, and spaces
            tweet_emojis_list = list(tweet_emojis.strip('[]').replace("\'", "").strip().split(","))
            emoji_counter = 0
            
            for emoji in tweet_emojis_list:           
                emoji_counter = emoji_counter + 1
            
            df.at[i, 'emojis_num'] = emoji_counter
        else:
            df.at[i, 'emojis_num'] = 0
            
    return df

#creates emoticons_num column
def create_emoticons_num(df):
    df['emoticons_num'] = 0 
    
    for i, row in df.iterrows():    
        if df.at[i, 'emoticons_flag']:
            tweet_emoticons = df.at[i, 'emoticons']
            #strip brackets, quote, and spaces
            tweet_emoticons_list = list(tweet_emoticons.strip('[]').replace("\'", "").strip().split(","))
            emoticons_counter = 0
            
            for emoticon in tweet_emoticons_list:            
                emoticons_counter = emoticons_counter + 1
            
            df.at[i, 'emoticons_num'] = emoticons_counter
        else:
            df.at[i, 'emoticons_num'] = 0
            
    return df

#creates hashtag_num column
def create_hashtags_num (df):
    df['hashtags_num'] = 0 
    
    for i, row in df.iterrows():    
        if df.at[i, 'hashtags_flag']:
            tweet_hashtags = df.at[i, 'hashtags']
            #strip brackets, quote, and spaces
            tweet_hashtags_list = list(tweet_hashtags.strip('[]').replace("\'", "").strip().split(","))
            hashtags_counter = 0
            
            for hashtag in tweet_hashtags_list:            
                hashtags_counter = hashtags_counter + 1
            
            df.at[i, 'hashtags_num'] = hashtags_counter
        else:
            df.at[i, 'hashtags_num'] = 0
            
    return df

In [4]:
df_tweets_cleaned = create_emojis_num(df_tweets_cleaned)
df_tweets_cleaned = create_emoticons_num(df_tweets_cleaned)
df_tweets_cleaned = create_hashtags_num(df_tweets_cleaned)

In [5]:
#df_tweets_cleaned.loc[df_tweets_cleaned['hashtags_flag'] == True]

### Generate columns `numbers_flag`, `numbers_num`
Generate a binary flag and a count of how many times numbers were used in a tweet. Numbers can either be numeric, or in English. English numbers are sometimes considered stop words by Spacy (e.g. "twelve" is a stop word in tweet `568911315026063361`, but "thirty" is not for some reason in tweet `568237684277141504`), and were removed in `lemmas_list`, so we generate the numbers features from column `text_cleaned_no_abbreviations`. We will use Spacy model to help us determine which token are like numbers, using `like_num`.

In [6]:
df_tweets_cleaned.loc[df_tweets_cleaned['tweet_id'] == 568911315026063361]

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,text,text_cleaned,text_cleaned_time_removed,emojis_flag,emojis,emoticons_flag,emoticons,text_cleaned_without_emojis_emoticons,hashtags,text_cleaned_without_emojis_emoticons_hashtags,hashtags_flag,text_cleaned_lower_case,text_cleaned_no_abbreviations,text_list_no_stop_words,lemmas_list,emojis_num,emoticons_num,hashtags_num
2767,568911315026063361,negative,1.0,Late Flight,1.0,United,@united 5.5 hours Late Flightr I've been in tr...,5.5 hours Late Flightr I've been in transit f...,5.5 hours Late Flightr I've been in transit f...,False,[],False,[],5.5 hours Late Flightr I've been in transit f...,[],5.5 hours Late Flightr I've been in transit f...,False,5.5 hours late flightr i've been in transit f...,5.5 hours late flightr i've been in transit f...,hours late flightr transit total hours change ...,hour late flightr transit total hour change pl...,0,0,0


In [7]:
#load spacy model
import spacy

nlp = spacy.load('en_core_web_md')

In [8]:
#this function will create the columns numbers_flag and numbers_num
def create_numbers_columns(df):
    df['numbers_flag'] = False
    df['numbers_num'] = 0
    
    for i, row in df.iterrows():   
        if i % 1000 == 0:
            print('at row number: ' + str(i))

        text = df.at[i, 'text_cleaned_no_abbreviations']
        #print(type(text))

        like_num_count = 0
        
        #tokenize text into list of tokens
        #print(text)
        
        token_list = nlp(text)

        #iterate through our tokens and count the number of nums
        for token in token_list:
            #print(token)
            if token.like_num:
                like_num_count = like_num_count + 1

        #at the end, we set our new columns
        if like_num_count != 0:
            df.at[i, 'numbers_flag'] = True
            df.at[i, 'numbers_num'] = like_num_count            
    
    return df

In [9]:
#Sanity check

create_numbers_columns(df_tweets_cleaned.loc[df_tweets_cleaned['tweet_id'] == 568911315026063361])
#create_numbers_columns(df_tweets_cleaned.loc[df_tweets_cleaned['tweet_id'] == 570093964059156481])

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,text,text_cleaned,text_cleaned_time_removed,emojis_flag,emojis,emoticons_flag,emoticons,text_cleaned_without_emojis_emoticons,hashtags,text_cleaned_without_emojis_emoticons_hashtags,hashtags_flag,text_cleaned_lower_case,text_cleaned_no_abbreviations,text_list_no_stop_words,lemmas_list,emojis_num,emoticons_num,hashtags_num,numbers_flag,numbers_num
2767,568911315026063361,negative,1.0,Late Flight,1.0,United,@united 5.5 hours Late Flightr I've been in tr...,5.5 hours Late Flightr I've been in transit f...,5.5 hours Late Flightr I've been in transit f...,False,[],False,[],5.5 hours Late Flightr I've been in transit f...,[],5.5 hours Late Flightr I've been in transit f...,False,5.5 hours late flightr i've been in transit f...,5.5 hours late flightr i've been in transit f...,hours late flightr transit total hours change ...,hour late flightr transit total hour change pl...,0,0,0,True,3


In [10]:
df_tweets_cleaned = create_numbers_columns(df_tweets_cleaned)

at row number: 0
at row number: 1000
at row number: 2000
at row number: 3000
at row number: 4000
at row number: 5000
at row number: 6000
at row number: 7000
at row number: 8000
at row number: 9000
at row number: 10000
at row number: 11000
at row number: 12000
at row number: 13000
at row number: 14000


In [11]:
#df_tweets_cleaned.loc[df_tweets_cleaned['numbers_flag'] == True]

### Generate columns `char_length_original`, `char_length_user`
Generate columns with the number of characters in original tweet, and cleaned tweet from column `text_cleaned`.

In [12]:
#this function will create the columns numbers_flag and numbers_num
def create_char_length_columns(df):
    df['char_length_original'] = 0
    df['char_length_user'] = 0
    
    for i, row in df.iterrows():
        text = df.at[i, 'text']
        cleaned_text = df.at[i, 'text_cleaned_no_abbreviations']
        
        df.at[i, 'char_length_original'] = len(text)
        df.at[i, 'char_length_user'] = len(cleaned_text)
    
    return df

In [13]:
df_tweets_cleaned = create_char_length_columns(df_tweets_cleaned)

### Generate columns `mentions_num`, `retweet_flag`, and `http_flag`
Generate columns `mentions_num`: number of mentions in a tweet, `retweet_flag`: whether a tweet has a retweet, and `http_flag`: whether a tweet has a http link.

In [14]:
import re

#this function will create mentions_num column
def create_mentions_num(df):
    df['mentions_num'] = 0
    
    for i, row in df.iterrows():
        text = df.at[i, 'text']
        regex_to_find = r'\@[\w\d]*'
        
        regex_hits_list = re.findall(regex_to_find, text)
        df.at[i, 'mentions_num'] = len(regex_hits_list)
    
    return df

#this function will create retweet_flag column
def create_retweet_flag(df):
    df['retweet_flag'] = False
    
    for i, row in df.iterrows():
        text = df.at[i, 'text']
        regex_to_find = r'RT \@.*'
        
        regex_hits_list = re.findall(regex_to_find, text)
        if (len(regex_hits_list) != 0):
            df.at[i, 'retweet_flag'] = True
    
    return df

#this function will create http_flag column
def create_http_flag(df):
    df['http_flag'] = False
    
    for i, row in df.iterrows():
        text = df.at[i, 'text']
        regex_to_find = r'https*://[^\s]*'
        
        regex_hits_list = re.findall(regex_to_find, text)
        if (len(regex_hits_list) != 0):
            df.at[i, 'http_flag'] = True
    
    return df

In [15]:
df_tweets_cleaned = create_mentions_num(df_tweets_cleaned)
df_tweets_cleaned = create_retweet_flag(df_tweets_cleaned)
df_tweets_cleaned = create_http_flag(df_tweets_cleaned)

### Scale numeric columns
The numeric columns will likely need to be scaled to a scale from 0 to 1. For columns `char_length_original` and `char_length_user` we will use normal MinMaxScaler because there aren't any big outliers, but for the other columns we will use RobustScaler as there are outliers.

In [16]:
df_tweets_cleaned[['emojis_num', 'emoticons_num', 'hashtags_num', 'numbers_num', 'char_length_original', 'char_length_user', 'mentions_num']].describe()

,emojis_num,emoticons_num,hashtags_num,numbers_num,char_length_original,char_length_user,mentions_num
count,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000
mean,0.066667,0.019262,0.238525,0.426571,103.836066,88.200068,1.132719
std,0.612111,0.140400,0.654195,0.738649,36.281706,36.835608,0.410359
min,0.000000,0.000000,0.000000,0.000000,12.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,77.000000,59.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,114.000000,96.000000,1.000000
75%,0.000000,0.000000,0.000000,1.000000,136.000000,121.000000,1.000000
max,40.000000,3.000000,8.000000,7.000000,186.000000,177.000000,6.000000


In [17]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler

minMaxScaler = MinMaxScaler()
df_tweets_cleaned['char_length_original_scaled'] = 0
df_tweets_cleaned['char_length_user_scaled'] = 0
df_tweets_cleaned[['char_length_original_scaled', 'char_length_user_scaled']] = \
    minMaxScaler.fit_transform(df_tweets_cleaned[['char_length_original', 'char_length_user']])

robustScaler = RobustScaler()
df_tweets_cleaned['emojis_num_scaled'] = 0
df_tweets_cleaned['emoticons_num_scaled'] = 0
df_tweets_cleaned['hashtags_num_scaled'] = 0
df_tweets_cleaned['numbers_num_scaled'] = 0
df_tweets_cleaned['mentions_num_scaled'] = 0
df_tweets_cleaned[['emojis_num_scaled', 'emoticons_num_scaled', 'hashtags_num_scaled', 'numbers_num_scaled', 'mentions_num_scaled']] = \
    minMaxScaler.fit_transform(df_tweets_cleaned[['emojis_num', 'emoticons_num', 'hashtags_num', 'numbers_num', 'mentions_num']])

### Convert binary True/False columns to 1s/0s
We should convert binary True/False columns to 1s/0s.

In [18]:
df_tweets_cleaned['emojis_flag'] = df_tweets_cleaned['emojis_flag'].astype(int)
df_tweets_cleaned['emoticons_flag'] = df_tweets_cleaned['emoticons_flag'].astype(int)
df_tweets_cleaned['hashtags_flag'] = df_tweets_cleaned['hashtags_flag'].astype(int)
df_tweets_cleaned['numbers_flag'] = df_tweets_cleaned['numbers_flag'].astype(int)
df_tweets_cleaned['retweet_flag'] = df_tweets_cleaned['retweet_flag'].astype(int)
df_tweets_cleaned['http_flag'] = df_tweets_cleaned['http_flag'].astype(int)

### Group the positive and neutral 
As stated before, our goal is to predict negative sentiment tweets; we don't particularly care if the tweets are positive or neutral, to us they are the same thing: not negative. Therefore, we merge the positive and neutral classes to 0s, and rename negative label to 1s.

In [19]:
df_tweets_cleaned['binary_response_variable'] = False

df_tweets_cleaned.loc[df_tweets_cleaned.airline_sentiment == 'neutral', 'binary_response_variable'] = False
df_tweets_cleaned.loc[df_tweets_cleaned.airline_sentiment == 'positive', 'binary_response_variable'] = False
df_tweets_cleaned.loc[df_tweets_cleaned.airline_sentiment == 'negative', 'binary_response_variable'] = True

In [20]:
df_tweets_cleaned.binary_response_variable

0        False
1        False
2        False
3         True
4         True
         ...  
14635    False
14636     True
14637    False
14638     True
14639    False
Name: binary_response_variable, Length: 14640, dtype: bool

In [21]:
df_tweets_cleaned.binary_response_variable = df_tweets_cleaned.binary_response_variable.astype(int)

In [22]:
df_tweets_cleaned

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,text,text_cleaned,text_cleaned_time_removed,emojis_flag,emojis,emoticons_flag,emoticons,text_cleaned_without_emojis_emoticons,hashtags,text_cleaned_without_emojis_emoticons_hashtags,hashtags_flag,text_cleaned_lower_case,text_cleaned_no_abbreviations,text_list_no_stop_words,lemmas_list,emojis_num,emoticons_num,hashtags_num,numbers_flag,numbers_num,char_length_original,char_length_user,mentions_num,retweet_flag,http_flag,char_length_original_scaled,char_length_user_scaled,emojis_num_scaled,emoticons_num_scaled,hashtags_num_scaled,numbers_num_scaled,mentions_num_scaled,binary_response_variable
0,570306133677760513,neutral,1.0000,,,Virgin America,@VirginAmerica What @dhepburn said.,What said.,What said.,0,[],0,[],What said.,[],What said.,0,what said.,what said.,said,say,0,0,0,0,0,35,12,2,0,0,0.132184,0.067797,0.0,0.0,0.000,0.000000,0.2,0
1,570301130888122368,positive,0.3486,,0.0,Virgin America,@VirginAmerica plus you've added commercials t...,plus you've added commercials to the experien...,plus you've added commercials to the experien...,0,[],0,[],plus you've added commercials to the experien...,[],plus you've added commercials to the experien...,0,plus you've added commercials to the experien...,plus you've added commercials to the experien...,plus added commercials experience tacky,plus add commercial experience tacky,0,0,0,0,0,72,58,1,0,0,0.344828,0.327684,0.0,0.0,0.000,0.000000,0.0,0
2,570301083672813571,neutral,0.6837,,,Virgin America,@VirginAmerica I didn't today... Must mean I n...,I didn't today... Must mean I need to take an...,I didn't today... Must mean I need to take an...,0,[],0,[],I didn't today... Must mean I need to take an...,[],I didn't today... Must mean I need to take an...,0,i didn't today... must mean i need to take an...,i didn't today... must mean i need to take an...,today mean need trip,today mean need trip,0,0,0,0,0,71,57,1,0,0,0.339080,0.322034,0.0,0.0,0.000,0.000000,0.0,0
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,@VirginAmerica it's really aggressive to blast...,"it's really aggressive to blast obnoxious ""en...","it's really aggressive to blast obnoxious ""en...",0,[],0,[],"it's really aggressive to blast obnoxious ""en...",[],"it's really aggressive to blast obnoxious ""en...",0,"it's really aggressive to blast obnoxious ""en...","it's really aggressive to blast obnoxious ""en...",aggressive blast obnoxious entertainment guest...,aggressive blast obnoxious entertainment guest...,0,0,0,0,0,126,108,1,0,0,0.655172,0.610169,0.0,0.0,0.000,0.000000,0.0,1
4,570300817074462722,negative,1.0000,Can't Tell,1.0,Virgin America,@VirginAmerica and it's a really big bad thing...,and it's a really big bad thing about it,and it's a really big bad thing about it,0,[],0,[],and it's a really big bad thing about it,[],and it's a really big bad thing about it,0,and it's a really big bad thing about it,and it's a really big bad thing about it,big bad thing,big bad thing,0,0,0,0,0,55,41,1,0,0,0.247126,0.231638,0.0,0.0,0.000,0.000000,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,,0.0,American,@AmericanAir thank you we got on a different f...,thank you we got on a different flight to Chi...,thank you we got on a different flight to Chi...,0,[],0,[],thank you we got on a different flight to Chi...,[],thank you we got on a different flight to Chi...,0,thank you we got on a different flight to chi...,thank you we got on a different flight to chi...,thank got different flight chicago,thank get different flight chicago,0,0,0,0,0,63,51,1,0,0,0.293103,0.288136,0.0,0.0,0.000,0.000000,0.0,0
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0,American,@AmericanAir leaving over 20 minutes Late Flig...,leaving over 20 minutes Late F

### Get rid of all other columns
We only need the `binary_response_variable`, `_flag` columns, `_scaled` columns, and `lemmas_list` column (this will be vectorized using our models).

In [23]:
df_tweets_cleaned.columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline', 'text',
       'text_cleaned', 'text_cleaned_time_removed', 'emojis_flag', 'emojis',
       'emoticons_flag', 'emoticons', 'text_cleaned_without_emojis_emoticons',
       'hashtags', 'text_cleaned_without_emojis_emoticons_hashtags',
       'hashtags_flag', 'text_cleaned_lower_case',
       'text_cleaned_no_abbreviations', 'text_list_no_stop_words',
       'lemmas_list', 'emojis_num', 'emoticons_num', 'hashtags_num',
       'numbers_flag', 'numbers_num', 'char_length_original',
       'char_length_user', 'mentions_num', 'retweet_flag', 'http_flag',
       'char_length_original_scaled', 'char_length_user_scaled',
       'emojis_num_scaled', 'emoticons_num_scaled', 'hashtags_num_scaled',
       'numbers_num_scaled', 'mentions_num_scaled',
       'binary_response_variable'],
      dtype='object')

In [24]:
df_tweets_cleaned = df_tweets_cleaned[[
    'binary_response_variable', 
    'emojis_flag', 
    'emoticons_flag',
    'hashtags_flag', 
    'numbers_flag',
    'retweet_flag',
    'http_flag',
    'char_length_original_scaled',
    'char_length_user_scaled',
    'emojis_num_scaled',
    'emoticons_num_scaled',
    'hashtags_num_scaled',
    'numbers_num_scaled',
    'mentions_num_scaled',
    'lemmas_list'
]]

## Baseline Benchmark with trained word2vec gensim model
We do a baseline benchmark with Patrick's trained word2vec gensim model, located at `..\milestone1\Patrick\gensim_word2vec_trained.bin` of this repo.

### Split into train and test datasets
We need to split our dataframe into train and test datasets/dataframes. We don't need a validation set for now, but we will split into one later for our machine learning model tuning. For now, 70%/30% for train/test seems good enough. We will split based on column `binary_response_variable`.

In [25]:
from sklearn.model_selection import train_test_split

X = df_tweets_cleaned.loc[:, df_tweets_cleaned.columns != 'binary_response_variable']
Y = df_tweets_cleaned.loc[:, df_tweets_cleaned.columns == 'binary_response_variable']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1, stratify=Y)

### Generate Tweet vectors

In [26]:
import gensim

custom_w2v_gensim_model_path = '..\milestone1\Patrick\gensim_word2vec_trained.bin'
w2v_gensim_model = gensim.models.KeyedVectors.load_word2vec_format(custom_w2v_gensim_model_path, binary=True)

In [27]:
#calculates a vector for a given Tweet
def calculate_average_tweet_vector(tweet, w2v_model, num_dimensions):
    tokens = tweet.split(' ')
    
    tweet_vector = np.zeros(num_dimensions, np.float32)
    actual_token_count = 0
    
    for token in  tokens:
        if token in w2v_model.wv.vocab:
            actual_token_count = actual_token_count + 1
            tweet_vector = np.add(tweet_vector, w2v_model[token])
        
    tweet_vector = np.divide(tweet_vector, actual_token_count)
    
    return tweet_vector

In [28]:
#Sanity check
calculate_average_tweet_vector("arrangement reimburse rental", w2v_gensim_model, w2v_gensim_model.wv.vector_size)

array([-1.355661  , -0.0425607 ,  0.08605277,  0.77414465, -0.18937008,
       -0.5402817 ,  0.683446  ,  0.37354645,  0.73340803, -0.12201399,
        0.2240843 , -0.00996478, -0.18526036,  0.24696898,  0.12944683,
       -0.43216953,  0.37981465, -0.3665507 ,  0.7351089 ,  0.6667984 ,
        0.2898368 , -0.91822857, -0.5228736 ,  0.21097268,  0.04620714,
        0.2920395 , -0.56135315,  0.8466854 , -0.09413695, -0.0781236 ,
       -0.2157198 , -0.42463648, -0.2514145 ,  0.1562465 , -0.04070664,
       -0.3206952 ,  0.72867155, -0.05172675,  0.746311  , -0.44089374,
        0.04898795, -0.06308773, -0.02671836, -0.5343575 , -0.84297615,
        0.31838188,  0.04328161,  0.21963978, -0.9148485 ,  0.3306698 ,
       -0.56024903,  0.30039105,  1.2359936 , -0.11688393,  1.2390062 ,
        0.32984343,  0.16735716, -0.3419783 , -0.31109077, -0.23789054,
       -0.64596885,  0.5859638 , -0.3904748 ,  0.50065964, -0.7277306 ,
       -0.58542424,  0.8771445 , -0.13310944, -0.25320467, -0.41

In [29]:
num_dimensions = w2v_gensim_model.wv.vector_size
X_train['tweet_vector'] = X_train.lemmas_list.apply(lambda text: calculate_average_tweet_vector(text, w2v_gensim_model, num_dimensions))
X_test['tweet_vector'] = X_test.lemmas_list.apply(lambda text: calculate_average_tweet_vector(text, w2v_gensim_model, num_dimensions))

In [30]:
df_train_tweet_vector = pd.DataFrame(list(X_train['tweet_vector']), index=X_train.index)
df_test_tweet_vector = pd.DataFrame(list(X_test['tweet_vector']), index=X_test.index)

In [31]:
#after creating new dataframes above, any zeroes are converted to null...we need to convert them back to zeroes...
df_train_tweet_vector = df_train_tweet_vector.fillna(0)
df_test_tweet_vector = df_test_tweet_vector.fillna(0)

In [32]:
X_train_combined = pd.concat([df_train_tweet_vector, X_train], axis=1)
X_test_combined = pd.concat([df_test_tweet_vector, X_test], axis=1)

### Remove `tweet_vector` and `lemmas_list` column

In [33]:
X_train_combined = X_train_combined.loc[:, X_train_combined.columns != 'tweet_vector']
X_test_combined = X_test_combined.loc[:, X_test_combined.columns != 'tweet_vector']
X_train_combined = X_train_combined.loc[:, X_train_combined.columns != 'lemmas_list']
X_test_combined = X_test_combined.loc[:, X_test_combined.columns != 'lemmas_list']

In [34]:
from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression(C=100)
logmodel.fit(X_train_combined, Y_train)
predictions = logmodel.predict(X_test_combined)

In [35]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,predictions))

              precision    recall  f1-score   support

           0       0.79      0.71      0.75      1639
           1       0.84      0.89      0.86      2753

    accuracy                           0.82      4392
   macro avg       0.81      0.80      0.80      4392
weighted avg       0.82      0.82      0.82      4392



## Output engineered features to CSV
Output the engineered features with gensim trained word embedding vectors to a csv

In [36]:
X_train_and_test = pd.concat([X_train_combined, X_test_combined], axis=0)
Y_train_and_test = pd.concat([Y_train, Y_test], axis=0)
all_features_df = pd.concat([X_train_and_test, Y_train_and_test], axis=1)
all_features_df.to_csv('..\\data\\gensim_word2vec_trained_with_engineered_features.csv')

## Baseline Benchmark with Google word2vec gensim model subset
We do a baseline benchmark with Google's word2vec gensim model that is subset with our word vectors as well, located at `..\milestone1\Patrick\gensim_word2vec_google_subset.bin`, which can be unzipped from `..\milestone1\Patrick\gensim_word2vec_google_subset.bin` of this repo.

### Split into train and test datasets
We need to split our dataframe into train and test datasets/dataframes. We don't need a validation set for now, but we will split into one later for our machine learning model tuning. For now, 70%/30% for train/test seems good enough. We will split based on column `binary_response_variable`.

In [37]:
from sklearn.model_selection import train_test_split

X = df_tweets_cleaned.loc[:, df_tweets_cleaned.columns != 'binary_response_variable']
Y = df_tweets_cleaned.loc[:, df_tweets_cleaned.columns == 'binary_response_variable']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1, stratify=Y)

### Generate Tweet vectors

In [38]:
import gensim

custom_w2v_gensim_model_path = '..\milestone1\Patrick\gensim_word2vec_google_subset.bin'
w2v_gensim_model = gensim.models.KeyedVectors.load_word2vec_format(custom_w2v_gensim_model_path, binary=True)

In [39]:
#calculates a vector for a given Tweet
def calculate_average_tweet_vector(tweet, w2v_model, num_dimensions):
    tokens = tweet.split(' ')
    
    tweet_vector = np.zeros(num_dimensions, np.float32)
    actual_token_count = 0
    
    for token in  tokens:
        if token in w2v_model.wv.vocab:
            actual_token_count = actual_token_count + 1
            #print(type(w2v_model[token]))
            tweet_vector = np.add(tweet_vector, w2v_model[token])
        
    tweet_vector = np.divide(tweet_vector, actual_token_count)
    
    #print(type(tweet_vector))
    
    return tweet_vector

In [40]:
#Sanity check
calculate_average_tweet_vector("arrangement reimburse rental", w2v_gensim_model, w2v_gensim_model.wv.vector_size)

array([ 3.54003906e-02, -1.12874351e-01, -2.18098965e-02,  9.09016952e-02,
       -2.27050781e-02, -8.56933594e-02,  7.82063827e-02,  2.92968750e-03,
        1.50634766e-01, -5.30598946e-02, -2.09147129e-02,  4.85026054e-02,
       -7.74739608e-02,  3.61328125e-02, -1.35253906e-01,  2.02148438e-01,
        1.25406906e-01,  1.09863281e-02, -9.44010448e-03,  8.39843750e-02,
        1.71671554e-01,  9.71679688e-02,  9.78597030e-02,  1.00423180e-01,
       -7.32421875e-03,  5.04557304e-02, -2.04589844e-01, -4.00390625e-02,
        3.33658862e-03,  5.76171875e-02, -1.45914719e-01, -1.64225265e-01,
       -6.94580078e-02,  1.59505203e-01,  2.04345703e-01, -1.22558594e-01,
        1.31835938e-02,  3.41796875e-03, -9.76562500e-02, -8.54492188e-02,
        7.74739608e-02,  1.63981125e-01,  1.13444008e-01, -2.68229157e-01,
       -2.82552093e-01, -1.71610519e-01,  9.11051407e-02, -5.90820312e-02,
       -1.35172531e-01,  1.21815996e-02,  2.30631515e-01, -8.55306014e-02,
        3.49121094e-02, -

In [41]:
num_dimensions = w2v_gensim_model.wv.vector_size
X_train['tweet_vector'] = X_train.lemmas_list.apply(lambda text: calculate_average_tweet_vector(text, w2v_gensim_model, num_dimensions))
X_test['tweet_vector'] = X_test.lemmas_list.apply(lambda text: calculate_average_tweet_vector(text, w2v_gensim_model, num_dimensions))

In [42]:
df_train_tweet_vector = pd.DataFrame(list(X_train['tweet_vector']), index=X_train.index)
df_test_tweet_vector = pd.DataFrame(list(X_test['tweet_vector']), index=X_test.index)

In [43]:
#after creating new dataframes above, any zeroes are converted to null...we need to convert them back to zeroes...
df_train_tweet_vector = df_train_tweet_vector.fillna(0)
df_test_tweet_vector = df_test_tweet_vector.fillna(0)

In [44]:
X_train_combined = pd.concat([df_train_tweet_vector, X_train], axis=1)
X_test_combined = pd.concat([df_test_tweet_vector, X_test], axis=1)

### Remove `tweet_vector` and `lemmas_list` column

In [45]:
X_train_combined = X_train_combined.loc[:, X_train_combined.columns != 'tweet_vector']
X_test_combined = X_test_combined.loc[:, X_test_combined.columns != 'tweet_vector']
X_train_combined = X_train_combined.loc[:, X_train_combined.columns != 'lemmas_list']
X_test_combined = X_test_combined.loc[:, X_test_combined.columns != 'lemmas_list']

In [46]:
from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression(C=100)
logmodel.fit(X_train_combined, Y_train)
predictions = logmodel.predict(X_test_combined)

In [47]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,predictions))

              precision    recall  f1-score   support

           0       0.79      0.70      0.74      1639
           1       0.83      0.89      0.86      2753

    accuracy                           0.82      4392
   macro avg       0.81      0.79      0.80      4392
weighted avg       0.82      0.82      0.82      4392



In [48]:
predictions

array([1, 1, 1, ..., 0, 1, 0])

## Output engineered features to CSV
Output the engineered features with Google's word2vec gensim trained word embedding vectors to a csv

In [49]:
X_train_and_test = pd.concat([X_train_combined, X_test_combined], axis=0)
Y_train_and_test = pd.concat([Y_train, Y_test], axis=0)
all_features_df = pd.concat([X_train_and_test, Y_train_and_test], axis=1)
all_features_df.to_csv('..\\data\\gensim_word2vec_google_subset_with_engineered_features.csv')

## Baseline Benchmark with GloVe Word Embeddings
We do a baseline benchmark with a GloVe word embedding built with the en_vectors_web_lg model from https://spacy.io/models/en-starters#en_vectors_web_lg.

### Split into train and test datasets
We need to split our dataframe into train and test datasets/dataframes. We don't need a validation set for now, but we will split into one later for our machine learning model tuning. For now, 70%/30% for train/test seems good enough. We will split based on column `binary_response_variable`.

In [50]:
from sklearn.model_selection import train_test_split

X = df_tweets_cleaned.loc[:, df_tweets_cleaned.columns != 'binary_response_variable']
Y = df_tweets_cleaned.loc[:, df_tweets_cleaned.columns == 'binary_response_variable']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1, stratify=Y)

### Load up GloVe embeddings for our data set

In [51]:
glove_embed = pd.read_csv('..\milestone1\Patrick\glove_embeddings.csv', encoding='utf-8', na_filter= False)

glove_embed = glove_embed.set_index(['word'])
glove_embed
#glove_embed.drop(['word'], axis='columns')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
word,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
reinforcement,-0.081670,0.813500,0.023805,-0.085249,-0.230950,-0.303480,-0.269250,0.435900,-0.290920,0.29710,-0.30387,0.305880,0.213880,0.535730,-0.353490,0.648360,0.160050,1.44500,-0.271980,0.143200,-0.192130,-0.251260,0.559250,0.528460,-0.43528,0.189720,0.166900,-0.177370,-0.151070,-0.18809,-0.061047,0.041127,-0.187210,0.443370,0.078184,0.153870,-0.351450,-0.230860,-0.146670,-0.70027,0.075809,0.135820,-0.183040,0.024308,0.006076,-0.487580,-0.399980,0.446720,-0.22531,-0.522430,0.755120,0.006996,-0.267220,0.545480,-0.499780,0.831870,0.449690,0.671700,-0.28061,0.882930,-0.326870,0.174510,0.17002,0.161590,0.291800,-0.026018,0.209620,0.155010,-0.688490,-0.234970,-0.965750,-0.16768,0.261540,-0.37860,-0.066713,0.381840,0.142430,0.24210,-0.315170,0.074694,0.212300,-0.378330,-0.186910,-0.563640,0.027921,0.13485,0.165150,0.62586,0.31915,0.538670,0.32050,-0.440860,-0.209190,0.161200,-0.327470,0.161840,0.567800,0.203240,0.391930,0.074044,0.219920,-0.512320,-0.153290,0.213790,0.129960,-2.26450,0.348350,0.655450,-0.504910,0.057695,0.073712,-0.083758,0.36849,0.24195,0.281010,-0.288140,0.047334,-0.032487,-0.17019,0.107240,-0.375790,0.069587,0.261510,0.758520,-0.171320,-0.054628,-0.256100,0.939910,0.045817,0.458330,0.519200,-0.060311,-0.470200,0.295380,0.681750,0.236330,-0.317660,-0.008197,-0.355710,-0.265690,-0.015033,0.354440,-0.074040,0.23583,-0.316270,-0.460530,0.120450,-0.243720,0.081290,-0.140620,-0.616910,0.086530,-0.661390,-0.25592,0.162270,0.303740,0.287370,-0.149120,0.093000,-0.20705,-0.064119,-0.306530,-0.123580,-0.17564,0.076782,-0.110810,0.128630,0.043224,-0.264390,-0.256760,0.97856,-0.470720,-0.322150,0.63500,0.070177,0.358630,0.196230,0.447910,0.62668,-0.251760,0.410470,0.260420,-0.005650,0.426430,-0.404030,-0.525130,0.255430,-0.336820,-0.862120,-0.563070,-0.053527,-0.582480,-0.211930,0.005291,-0.178480,0.324610,0.359230,-0.458880,-0.132210,0.417650,0.195240,0.130520,0.331060,-0.043837,0.571940,-0.013986,-0.309770,-0.387470,-0.244190,0.106330,-0.596610,0.543640,0.057847,-0.241900,0.58253,0.387150,-0.30910,-0.101300,0.005996,-0.425650,-0.36087,-0.27285,0.031300,-0.121310,-0.465340,0.262860,-0.399070,-0.631430,0.20592,0.333770,-0.407460,-0.186090,-0.305240,-0.125530,0.413730,-0.160030,0.222090,0.542320,0.262630,-0.028009,0.586860,-0.227670,-0.805640,0.041514,0.166120,-0.220660,-0.30448,-0.312170,-0.10486,-0.182820,-0.040229,0.402600,0.281300,0.433380,-0.369660,-0.043685,0.412990,-0.298570,-0.005131,0.361410,0.054864,-0.361950,-0.52791,-0.425010,-0.725350,0.767100,0.355960,0.135790,0.605910,0.217750,0.26006,0.003742,0.094306,-0.403860,-0.586990,-0.570700,0.5570

In [52]:
#glove_embed.loc[glove_embed.index ==  'arrangement'].to_numpy()[0]

### Generate Tweet vectors

In [53]:
#doc_glove_vectors = np.array([nlp(str(doc)).vector for doc in df_tweets_cleaned['lemmas_list'].values.astype('U')])

In [54]:
#pd.DataFrame(doc_glove_vectors).to_csv("glove_embeddings_document.csv")

In [55]:
#calculates a vector for a given Tweet
def calculate_average_tweet_vector_glove(tweet, num_dimensions):
    #print("tweet: " + tweet)
    tokens = tweet.split(' ')
    
    tweet_vector = np.zeros(num_dimensions, np.float64)
    actual_token_count = 0
    
    for token in  tokens:
        if token in glove_embed.index.values:
            #print (token)
            actual_token_count = actual_token_count + 1
            #print(type(glove_embed.loc[glove_embed.index ==  token]))
            #print(glove_embed.columns)
            #print(glove_embed.loc[glove_embed.index ==  token])
            
            tweet_vector = np.add(tweet_vector, glove_embed.loc[glove_embed.index ==  token].to_numpy()[0])
            #print('tweet_vector: '+ str(tweet_vector))

        
    tweet_vector = np.divide(tweet_vector, actual_token_count)
    
    #tweet_vector.reset_index(drop=True, inplace=True)
    #print(tweet_vector)
    
    #print(type(tweet_vector))
    
    return tweet_vector

In [56]:
#Sanity check
calculate_average_tweet_vector_glove("arrangement flight plane", 300)

array([ 2.83233333e-01, -2.55533333e-02,  9.44003333e-02,  5.85966667e-02,
        1.53080000e-01,  1.59166667e-01, -1.80970000e-01,  2.16610000e-01,
       -8.71376667e-02,  2.04333333e+00,  2.94066667e-02,  4.27506667e-01,
        2.22603333e-01,  4.82623333e-01, -2.02466000e-01, -2.99570000e-01,
       -2.68316667e-01,  1.69596667e+00,  4.60156667e-01,  1.82429333e-02,
       -4.56993333e-02,  3.82463333e-01, -4.06893333e-01,  9.15133333e-02,
        1.81127400e-02, -2.14426667e-01, -1.83620000e-01, -3.81030000e-01,
        8.41210000e-02,  7.60066667e-02, -2.30584000e-01,  1.21203667e-01,
        1.48416000e-01,  5.32806667e-01, -1.31336667e-01,  1.43090000e-01,
       -2.75850000e-03, -1.09174667e-01,  4.15940000e-02, -5.14143333e-01,
        3.47610000e-01, -1.17116667e-01,  1.00540000e-01,  1.77198033e-01,
        3.30304067e-01,  1.41640000e-01, -3.81560000e-01, -5.83700000e-02,
        3.38790000e-01,  6.96356667e-01,  8.06066667e-02, -5.39510000e-01,
       -2.18657333e-01,  

In [57]:
num_dimensions = 300
X_train['tweet_vector'] = X_train.lemmas_list.apply(lambda text: calculate_average_tweet_vector_glove(text, 300))
X_test['tweet_vector'] = X_test.lemmas_list.apply(lambda text: calculate_average_tweet_vector_glove(text, 300))

In [58]:
df_train_tweet_vector = pd.DataFrame(list(X_train['tweet_vector']), index=X_train.index)
df_test_tweet_vector = pd.DataFrame(list(X_test['tweet_vector']), index=X_test.index)

In [59]:
#after creating new dataframes above, any zeroes are converted to null...we need to convert them back to zeroes...
df_train_tweet_vector = df_train_tweet_vector.fillna(0)
df_test_tweet_vector = df_test_tweet_vector.fillna(0)

In [60]:
X_train_combined = pd.concat([df_train_tweet_vector, X_train], axis=1)
X_test_combined = pd.concat([df_test_tweet_vector, X_test], axis=1)

### Remove `tweet_vector` and `lemmas_list` column

In [61]:
X_train_combined = X_train_combined.loc[:, X_train_combined.columns != 'tweet_vector']
X_test_combined = X_test_combined.loc[:, X_test_combined.columns != 'tweet_vector']
X_train_combined = X_train_combined.loc[:, X_train_combined.columns != 'lemmas_list']
X_test_combined = X_test_combined.loc[:, X_test_combined.columns != 'lemmas_list']

In [62]:
X_train['tweet_vector']

4771     [0.04259503333333334, 0.1870333333333333, -0.1...
12454    [-0.3030950000000001, 0.3984211111111111, -0.2...
3840     [-0.007435000000000004, -0.093223625, -0.20564...
14176    [0.11595077777777776, 0.08231533333333335, 0.0...
1219     [0.08632333333333335, 0.09388633333333335, 0.0...
                               ...                        
7233     [0.055534999999999994, 0.12993475000000002, -0...
3855     [-0.06686600000000001, 0.0196266, -0.015812399...
6339     [-0.448548, 0.2620344, -0.49373199999999995, -...
11606    [0.08744272727272727, -0.0027381818181818137, ...
3899     [0.09479844545454545, 0.014179727272727278, 0....
Name: tweet_vector, Length: 10248, dtype: object

In [63]:
from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression(C=100)
logmodel.fit(X_train_combined, Y_train)
predictions = logmodel.predict(X_test_combined)

In [64]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,predictions))

              precision    recall  f1-score   support

           0       0.78      0.72      0.75      1639
           1       0.84      0.88      0.86      2753

    accuracy                           0.82      4392
   macro avg       0.81      0.80      0.80      4392
weighted avg       0.82      0.82      0.82      4392



In [65]:
predictions

array([0, 0, 1, ..., 0, 1, 0])

## Output engineered features to CSV
Output the engineered features with Glove word embedding vectors to a csv

In [66]:
X_train_and_test = pd.concat([X_train_combined, X_test_combined], axis=0)
Y_train_and_test = pd.concat([Y_train, Y_test], axis=0)
all_features_df = pd.concat([X_train_and_test, Y_train_and_test], axis=1)
all_features_df.to_csv('..\\data\\glove_word_embeddings_with_engineered_features.csv')